download

In [1]:
! pip install geemap

You should consider upgrading via the 'c:\users\hashim\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
! pip install -U -q earthengine-api

You should consider upgrading via the 'c:\users\hashim\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
! pip install -U -q folium

You should consider upgrading via the 'c:\users\hashim\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


import

In [4]:
import ee
import folium
import geemap
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

location selection

In [5]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWiK0jZlu6NAF8qkJSPV4XkhFnAHTWeBvtrTi00PLhsLrNzbjaITgzU

Successfully saved authorization token.


In [6]:
ee.Initialize()

In [7]:
#adding rastors from earth engine to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


#soecfy terrain area you want to segment (limited number of pixels)
bound_ = [[6.219817916110255,36.75729987187895]
,[6.421348372653224,36.75729987187895]
,[6.421348372653224,36.849939905468865]
,[6.219817916110255,36.849939905468865]
,[6.219817916110255,36.75729987187895]]



point_ = bound_[0]
point = ee.Geometry.Point(point_)
bound = ee.Geometry.Polygon(bound_)

importing satelite imagery

In [8]:
#importing the whole collection of satelite images
satlite_dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
#selecting the image with the best covrage for our location of interest
sat_image = satlite_dataset.filterBounds(point).sort('CLOUD_COVER').first()

In [9]:

# adding sat image to map
viz_params = {
    "min":0,
    "max":3000,
    "bands":["B4","B3","B2"]
}

map1 = folium.Map(location=[point_[1],point_[0]],zoom_start=12)
map1.add_ee_layer(sat_image.clip(bound),viz_params,name="sat_image")

In [10]:
map1

k-means clustering

In [11]:
#converting the rastor into a numpy array
rastor = geemap.ee_to_numpy(sat_image,region=bound,default_value=0)

In [12]:
w,h,f=rastor.shape
rastor = rastor.reshape((w*h,f))

In [13]:
#convert to pandas data-frame
#each pixel is row and each band is a column
data = pd.DataFrame(rastor,columns=sat_image.bandNames().getInfo())
data = data.drop(["sr_aerosol","pixel_qa","radsat_qa"],axis=1) #drop unused bands

In [14]:
#normlize data
data = minmax_scale(data)

In [15]:
#pca data reduction
n_components=5
pca = PCA(n_components)
data_r = pd.DataFrame(pca.fit_transform(data))

In [16]:
data_train = data_r.sample(frac=0.5)

In [17]:
#applying k-means
n_clusters = 4
km = KMeans(n_clusters,random_state=0)
km.fit(data_r)
classes = km.predict(data_r)

In [366]:
classes

array([2, 2, 2, ..., 2, 2, 2], dtype=int32)

In [18]:
classes = classes.reshape((w,h))

In [19]:
#classes coloring function
hue = {
    0:[82/255, 182/255, 154/255],
    1:[30/255, 96/255, 145/255],
    2:[217/255, 237/255, 146/255],
    3:[177/255, 167/255, 166/255]
}
f= lambda i:hue[i]

In [20]:
#adding k-means segmentation to the map 
map1 = folium.Map(location=[point_[1],point_[0]],zoom_start=12)
map1.add_ee_layer(sat_image.clip(bound),viz_params,name="sat_image")
folium.raster_layers.ImageOverlay(classes,bounds=[[bound_[2][1],bound_[2][0]],[bound_[0][1],bound_[0][0]]],colormap=f,name="sklearn cluster").add_to(map1)
folium.LayerControl().add_to(map1)
display(map1)
